<a href="https://colab.research.google.com/github/codingtolife/AI_Agriculture/blob/main/PlantDisease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
import opendatasets as od
from torchvision.datasets import ImageFolder
from torch import nn,optim
import torchvision.transforms as transform

In [3]:
url="https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset"
od.download(url,"./")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: sharmilakorada
Your Kaggle Key: ··········


100%|██████████| 2.70G/2.70G [00:14<00:00, 206MB/s]


In [4]:
transforms=transform.Compose([transform.Resize((64,64)),transform.RandomCrop((64,64)),transform.ToTensor()])
train_data=ImageFolder("/content/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train",transforms)
valid_data=ImageFolder("/content/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid",transforms)
train_loader=DataLoader(train_data,batch_size=10,shuffle=True)
valid_loader=DataLoader(valid_data,batch_size=10,shuffle=True)

In [5]:
len(train_data.classes)

38

In [6]:
for x,y in valid_loader:
  pass

In [7]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
import torchvision.models as models
model=models.resnet50(pretrained=True)
in_features=model.fc.in_features
model.fc=nn.Linear(in_features,len(train_data.classes))
model.to(device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
model(x.to(device)).shape

torch.Size([2, 38])

In [ ]:
def accuracy(y,y_hat):
  y_hat=list(y_hat)
  soft_y=[]
  for i in y_hat:
    i=list(i)
    soft_y.append(i.index(max(i)))
  crct=0
  y=list(y)
  for i in range(len(soft_y)):
    if(y[i]==soft_y[i]):
        crct=crct+1
  return crct/len(y)

epochs=20
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.01)
LOSS=[]
for epoch in range(epochs):
  l=0
  for x,y in train_loader:
    x,y=x.to(device),y.to(device)
    optimizer.zero_grad()
    y_hat=model(x)
    loss=criterion(y_hat,y)
    l=l+loss
    loss.backward()
    optimizer.step()
  LOSS.append(l)
  print(l)
  a=0
  idx=0
  for x,y in valid_loader:
    x,y=x.to(device),y.to(device)
    y_hat=model(x)
    acc=accuracy(y,y_hat)
    a=acc+a
    idx=idx+1
  print("accuracy:",a/idx)

tensor(14595.7520, device='cuda:0', grad_fn=<AddBackward0>)
accuracy: 0.6346416382252575
tensor(5926.9653, device='cuda:0', grad_fn=<AddBackward0>)
accuracy: 0.784926052332194
tensor(4039.8550, device='cuda:0', grad_fn=<AddBackward0>)
accuracy: 0.828156996587032
tensor(3162.0823, device='cuda:0', grad_fn=<AddBackward0>)
accuracy: 0.8610352673492621
tensor(2592.0042, device='cuda:0', grad_fn=<AddBackward0>)
accuracy: 0.8703640500568852
tensor(2217.5032, device='cuda:0', grad_fn=<AddBackward0>)
accuracy: 0.8889078498293553
tensor(1960.3514, device='cuda:0', grad_fn=<AddBackward0>)
accuracy: 0.8994880546075144
